In [97]:
#Import needed libraries

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

**Exercise 1**

In [98]:
# Loads dataset from specified path. Returns data matrix X and labels y
def load_Xy(path):

    #Use built-in pandas function to read csv file into a dataframe
    df = pd.read_csv(path)

    #Convert the dataframe to a numpy array
    data_array = df.to_numpy()

    #Separate the features X from the labels y
    X = data_array[:,:-1]
    y = data_array[:,-1]

    #Replace the 0 label with -1
    y[y==0] = -1 

    #reshape y as a column vector
    y = y.reshape(y.shape[0],1)

    #return X and y
    return X,y


In [99]:
#Call the function and load the data
X,y = load_Xy('data.csv')

In [100]:
#Let's count how many malicious apps there are

# Filter the y array only where y == -1, then get its dimensions, and then its length
y[ y==-1 ].shape[0]

14632

 There are $14632$ malicious apps in the dataset

In [101]:
#Count how many non-zero entries are in X
non_zero_entries = X[X!=0].shape[0]

#Count total elements in X
tot_entries = X.shape[0]*X.shape[1]

print("Non-zero entries: {}".format(non_zero_entries))
print("Total entries: {}".format(tot_entries))
print("Ratio: {}".format(non_zero_entries/tot_entries))

Non-zero entries: 277180
Total entries: 2522552
Ratio: 0.10988078739308446


Only the $10\%$ of the elements of the matrix $X$ are non-zero, thus $X$ has a sparsity of $90\%$.

In [102]:
#Returns the set of unique elements in the matrix X
np.unique(X)

array([0, 1], dtype=int64)

As we can see from the previous line of code, the matrix elements are either 1s or 0s. One hot coding is typically used to translate categorical features into multiple features that can only assume value 1 or 0. However, the features of the data matrix already assume values $\in \{0,1\}$, thus there is no need to apply one-hot coding.

**Exercise 2**

In [103]:
# Splits the data matrix X and the labels y into train and test set, according to ratio r.
def train_test_split(X,y,r):

    #Check that r is between 0 and 1
    if not ( r > 0 and r < 1):
        print("Error: r must be between 0 and 1")
        return
        
    #get number of rows
    rows = X.shape[0]

    #create a indices array for the rows
    indices = [i for i in range(rows)]

    #shuffle the array randomly
    np.random.shuffle(indices)

    #Now that the indices are randomized, we can split in train and test
    train_indices = indices[:int(rows*r)]
    test_indices = indices[int(rows*r):]

    X_train = X[train_indices,:]
    X_test = X[test_indices,:]
    y_train = y[train_indices]  
    y_test = y[test_indices]
    
    return X_train, y_train, X_test, y_test


In [104]:
#Split the data matrix into 50% train set and 50% test set

X_train, y_train, X_test, y_test = train_test_split(X,y,0.5)

**Exercise 3**

Let $X \in \R^{m \times n}$ be the data matrix with data columns $\textbf{x}_i,\ldots, \textbf{x}_n, i = 1,\ldots, n$, vector $\textbf{y} \in \{-1,1\}^n$, and a weight vector $\textbf{w} \in \R^m$. If the product $y_i \textbf{x}_i^\intercal \textbf{w} > 0$, the point is correctly classified, and if $y_i \textbf{x}_i^\intercal \textbf{w} <0$, the point is misclassified. We count the points $\textbf{x}_i^\intercal \textbf{w} = 0$ as misclassified, as there is no way of confirming its (mis)classification.

We define a function $f$ that assigns value $0$ to misclassified points, and $1$ for correctly classified points. Mathematically, this function would be defined as $f: \R \rightarrow \{0,1\}$:

$f(s) \coloneqq \frac{1}{2}(\text{sign}(s) + 1)$,

with $f(0) = 0$.

The function for the number of correctly classified points $F: \R^m \rightarrow \Z_{\geq 0}$ is then defined:

$F(\textbf{w}) = \sum_{i = 1} ^ n f(y_i\textbf{x}_i^\intercal \textbf{w})$.

Since Python does not have a sign function, we write functions that fulfill this task. 

We generate a random weight vector $w \in \R^m$ such that the chance of $w_j = 0$ is approximately zero, but we include a safety measure to stop the function in the case $\textbf{w}$ contains at least one $0$ element.

The functions for the sum of correctly classified points are defined as:

In [105]:
# Classifies the data points. Assigns labels +1 or -1
def classify(X,w):

    # For each row of X, compute x_i'*w
    v = np.dot(X,w)

    #if it's > 0, replace with +1
    v[v > 0] = 1

    #replace with -1 otherwise
    v[v <= 0] = -1
    return v

#Returns number of correctly classified datapoints using weight w
def count_correctly_classified(X,y,w):
    v = classify(X,w)*y
    return len(v[v == 1])

In [106]:
w_rand = np.random.rand(X.shape[1],1)*2 - 1

print("Correctly classified points: {}/{}".format(count_correctly_classified(X,y,w_rand), y.shape[0]))

Correctly classified points: 13448/29332


The number of correctly classified points for a random weights vector oscillates around $50 \%$

**Exercise 4**

$X \in \mathbb{R}^{m \times n}$, $w \in \mathbb{R}^{m \times 1}$, $y \in \mathbb{R}^{n \times 1}$

The Hinge-Loss function for logistic regression is:

$g(s) = \log{(1+e^{-s})}$.

As such, the cost function for logistic regression is defined as:

$J(\textbf{w}) = \sum_{i=1}^{n} \log{(1+e^{-y_i \textbf{x}_i^T \textbf{w}})} + \frac{\lambda}{2} \lVert \textbf{w} \rVert^2$,

with gradient:

$\nabla J (\textbf{w}) = \sum_{i=1}^{n} -\frac {e^{-y_i \textbf{x}_i^T \textbf{w}}}{1+e^{-y_i \textbf{x}_i^T \textbf{w}}}y_i\boldsymbol{x}_i + \lambda \textbf{w}$.

This gradient is determined as follows: consider the functions $g(s) = \log {(1+e^{-s})}$ and $h_i (\textbf{w}) = y_i \textbf{x}i^T \textbf{w}$. The cost function then becomes:

$J(\textbf{w}) = \sum_{i=1}^{n} g (h_i (\textbf{w})) + \frac{\lambda}{2}\lVert\textbf{w} \rVert^2$.

Note that the derivatives of $g$ and $h_i$ are defined as:

$g^\prime (s) = - \frac {e^{-s}}  {1+e^{-s}} = - \frac {1} {1+e^{s}}\quad$ , $\quad\nabla h_i (\textbf{w}) = y_i \textbf{x}_i$,

and trivially: $\frac{d}{d\textbf{w}}\left(\frac{\lambda}{2}\lVert\textbf{w} \rVert^2\right) = \lambda \textbf{w}$.

Then, due to the chain rule of the Jacobian, the gradient of the cost function becomes:

$\nabla J (\textbf{w}) = \sum_{i=1}^{n} g^\prime (y_i \textbf{x}_i^T \textbf{w}) \cdot \nabla h_i (\textbf{w}) + \lambda \textbf{w}$, 

obtaining the formula above.

**Exercise 5 - Normal**

In [107]:
# Derivative of f(s) = log(1 + e^(-s))
def fprime(x):
    return -1/(1 + np.exp(x))

In [108]:
# Computes the gradient of J(w) using the formula derived earlier
def grad(X,y,w,a,l):
    return np.dot( np.transpose(X), y*fprime(y*np.dot(X,w)) ) + l*w

# Computes the cost function J(w)
def loss(X,y,w,l):
    return np.sum( np.log(np.exp(-y*np.dot(X,w)) + 1 ) ) + (np.linalg.norm(w)**2)*0.5*l

#Logistic regression function. a (alpha) is step size, l (lambda) is regularization parameter.
#N is number of iterations

def lr(X,y,a,l,N):

    #Generate a random initial weight vector
    w = np.random.rand(86,1)

    #Perform fixed step gradient descent
    for i in range(N):
        g = grad(X,y,w,a,l)
        w = w - a*g
    
    return w

In [109]:
w_ = lr(X_train,y_train,0.00003,15000,50)

In [110]:
train_acc = count_correctly_classified(X_test,y_test,w_)/y_test.shape[0]
train_acc

0.8531978726305741

**Exercise 5 - Sparse**

In [111]:
#Uses sparse module from scipy library to further speed up computation
def grad_sparse(Xs,y,w,l):
    return  Xs.transpose()*(y*fprime(y*(Xs*w))) + l*w

def lr_sparse(X,y,a,l,N):
    #Converts data matrix into sparse format
    Xs = csr_matrix(X)

    #Generate random initial weight vector
    w = np.random.rand(86,1)

    for i in range(N):
        gradient = grad_sparse(Xs,y,w,l)
        w = w - a*gradient

    return w

In [112]:
# Different version of sparse logistic regression. 
# The algorithm stops either when
#   i) the norm of the gradient is smalled than threshold. This
#      indicates that the alogrithm has reached (almost) the minimum
#
#   ii) the number of iterations exceeds maxit
#
# Returns the weigth vector w and the number of iterations k
def lr_sparse_thr(X,y,a,l,threshold,maxit):
    Xs = csr_matrix(X)
    w = np.random.rand(86,1)

    #Ensures that at least the first iteration is performed
    grad_norm = threshold+1
    k = 0
    while grad_norm > threshold and k < maxit:
        g = grad_sparse(Xs,y,w,l)
        w = w - a*g
        grad_norm  = np.linalg.norm(g)
        k += 1

    return w,k

In this section we search for an optimal value of the regularization parameter $\lambda$. 
Sparse linear algebra allows us to perform a large number of iterations in a reasonable time.
We can then choose small $\alpha$, so that the optimal point is reached with precision.

We first look for $\lambda$ in a logarithmic scale, ranging from $10^{-6}$ to $10^{6}$, comparing the accuracy of the models on the train and test datasets

In [113]:
#choose small alpha and a large number of maximum iterations
alpha = 0.0001
maxit = 10000

#The algorithm stops when the norm of the gradient is smaller than grad_threshold
grad_threshold = 0.01

for lambda_ in [10**i for i in range(-6,6)]:
    w,k = lr_sparse_thr(X_train,y_train,alpha,lambda_,grad_threshold,maxit)
    train_acc = count_correctly_classified(X_train,y_train,w)/y_train.shape[0]
    test_acc = count_correctly_classified(X_test,y_test,w)/y_test.shape[0]
    print("Lambda: {}, iter: {}, train_acc: {}, test_acc: {}".format(lambda_, k, train_acc,test_acc))
    


Lambda: 1e-06, iter: 10000, train_acc: 0.9592254193372426, test_acc: 0.9580662757398064
Lambda: 1e-05, iter: 10000, train_acc: 0.9590890495022502, test_acc: 0.9582026455747988
Lambda: 0.0001, iter: 10000, train_acc: 0.9592254193372426, test_acc: 0.9579299059048139
Lambda: 0.001, iter: 10000, train_acc: 0.9590890495022502, test_acc: 0.9579980908223101
Lambda: 0.01, iter: 10000, train_acc: 0.9590890495022502, test_acc: 0.9580662757398064
Lambda: 0.1, iter: 10000, train_acc: 0.9592936042547389, test_acc: 0.9583390154097914
Lambda: 1, iter: 10000, train_acc: 0.9590890495022502, test_acc: 0.9590890495022502
Lambda: 10, iter: 7203, train_acc: 0.9571116868948588, test_acc: 0.9587481249147689
Lambda: 100, iter: 960, train_acc: 0.9498159007227601, test_acc: 0.9519978180826402
Lambda: 1000, iter: 118, train_acc: 0.9374062457384427, test_acc: 0.9379517250784126
Lambda: 10000, iter: 10000, train_acc: 0.5051138688122188, test_acc: 0.5061366425746625


<ipython-input-107-cf7674abd114>:3: RuntimeWarning: overflow encountered in exp
  return -1/(1 + np.exp(x))


Lambda: 100000, iter: 321, train_acc: 0.0, test_acc: 0.0


<ipython-input-111-9dc4296c0096>:3: RuntimeWarning: overflow encountered in multiply
  return  Xs.transpose()*(y*fprime(y*(Xs*w))) + l*w
<ipython-input-112-c5cbba0be9fd>:18: RuntimeWarning: invalid value encountered in subtract
  w = w - a*g


**Brute-force alpha**

In [114]:
#choose small alpha and a large number of maximum iterations
lambda_test = 50
maxit = 10000

#The algorithm stops when the norm of the gradient is smaller than grad_threshold
grad_threshold = 0.01

for alpha_ in [0.0004 + i*10**(-5) for i in range(-5,5)]:
    w,k = lr_sparse_thr(X_train,y_train,alpha_,lambda_test,grad_threshold,maxit)
    train_acc = count_correctly_classified(X_train,y_train,w)/y_train.shape[0]
    test_acc = count_correctly_classified(X_test,y_test,w)/y_test.shape[0]
    print("Alpha: {}, iter: {}, train_acc: {}, test_acc: {}".format(alpha_, k, train_acc,test_acc))
    


Alpha: 0.00035, iter: 505, train_acc: 0.9524069275876176, test_acc: 0.9532251465975726
Alpha: 0.00036, iter: 491, train_acc: 0.9524069275876176, test_acc: 0.9532251465975726
Alpha: 0.00037, iter: 484, train_acc: 0.9524069275876176, test_acc: 0.9532251465975726
Alpha: 0.00038, iter: 473, train_acc: 0.9524069275876176, test_acc: 0.9532251465975726
Alpha: 0.00039, iter: 443, train_acc: 0.9524069275876176, test_acc: 0.9532251465975726
Alpha: 0.0004, iter: 447, train_acc: 0.9524069275876176, test_acc: 0.9532251465975726
Alpha: 0.00041000000000000005, iter: 10000, train_acc: 0.9509068594027001, test_acc: 0.9522705577526251
Alpha: 0.00042, iter: 10000, train_acc: 0.9464066548479476, test_acc: 0.9474294286103914
Alpha: 0.00043000000000000004, iter: 10000, train_acc: 0.9388381290058639, test_acc: 0.9384290195008864
Alpha: 0.00044, iter: 10000, train_acc: 0.9366562116459839, test_acc: 0.9363152870585026


In [115]:
w,k = lr_sparse_thr(X_train,y_train,0.0001,0,0.01,10000)
train_acc = count_correctly_classified(X_train,y_train,w)/y_train.shape[0]
test_acc = count_correctly_classified(X_test,y_test,w)/y_test.shape[0]

print("{} {} {}".format(k,train_acc,test_acc))
    

10000 0.9590890495022502 0.9579980908223101


In [116]:
X_train_s = csr_matrix(X_train)
np.linalg.norm(grad_sparse(X_train_s,y_train,w,0))

4.704488950914988

In [117]:
w,k = lr_sparse_thr(X_train,y_train,0.00004,8500,0.0001,100000)

In [118]:
train_acc = count_correctly_classified(X_train,y_train,w)/y_train.shape[0]
test_acc = count_correctly_classified(X_test,y_test,w)/y_test.shape[0]

test_acc

0.8778808127642166

**Exercise 6**

In [119]:
def outlier_removal2 (path):
    #loading the data matrix from the file path
    [X,y] = load_Xy (path)

    #Use the singular value decomposition numpy function to calculate the matrices U, Vt and the vector S of the eigenvalues of X 
    [U, S, Vt] = np.linalg.svd (X, full_matrices = False)

    #Identify the most significant singular values taking the first k greater eigenvalues of S
    k = 80

    #Create the k-truncated svd matrices
    #Consider the matrix U_k taking the first k columns of U
    U_k = U [:,0:k]
    #Consider the matrix Vt_k taking the first k rows of Vt
    Vt_k = Vt [0:k,:]
    #Consider the diagonal matrix Sigma_k taking the first k eigenvalues of X
    Sigma_k = np.diag (S[0:k])

    #Calculating the approximate matrix of X, X_k 
    X_k = np.dot ( U_k,  np.dot (Sigma_k, Vt_k ))
    #X_k = np.dot ( U_k,  Vt_k )

    #percentage error of the approximate matrix X_k compared to the real matrix X
    p = np.linalg.norm (X-X_k, ord = 'fro') / np.linalg.norm (X, ord = 'fro')
    
    return [X_k, p]

[X_approx, n] = outlier_removal2 ('data2.csv')
n

0.05843407956007186

In [120]:
X_2,y_2 = load_Xy('data2.csv')
def k_truncated_SVD(X,t):
    X = X - np.mean(X, axis = 0)
    U, S, Vt = np.linalg.svd(X, full_matrices = False)
    k = np.where(S>t)[0]
    U_k = U[:,k]; Sigma_k = np.diag (S[k]); Vt_k = Vt_k = Vt [k,:]
    #print("{} {} {}".format(U_k.shape, Sigma_k.shape, Vt_k.shape))
    return len(k), np.dot(U_k, np.dot(Sigma_k, Vt_k))

def removal_error(X_1,X,t):
    k, X_k = k_truncated_SVD(X,t)
    X = X - np.mean(X, axis = 0)
    error = np.zeros(X_k.shape[0])
    error = np.linalg.norm(X - X_k, axis = 1, ord = 2)
    indices = np.argpartition(error, -2000)[-2000:]
    sort_ind = indices[np.argsort(-error[indices])]
    print(sort_ind)
    X_clean = np.delete(X_k, sort_ind, 0)
    #print(X_clean.shape, sort_ind.shape)
    return k, np.linalg.norm(X_clean - X_1, ord = 'fro')/np.linalg.norm(X_1, ord = 'fro')


for t in range(10,21):
    k, E = removal_error(X, X_2, t)
    print("Threshold: {}, Removal error: {}, k: {}".format(t, E, k))

[    1     8     6 ... 30148 11483   360]
Threshold: 10, Removal error: 1.1815185774436276, k: 86
[    1     8     6 ... 30148 11483   360]
Threshold: 11, Removal error: 1.1815185774436276, k: 86
[    1     8     6 ... 30148 11483   360]
Threshold: 12, Removal error: 1.1815185774436276, k: 86
[30650 30266 30999 ... 10617  5237 11202]
Threshold: 13, Removal error: 1.1297019684280325, k: 83
[31074 31201 31252 ... 30218 30580 30532]
Threshold: 14, Removal error: 1.1284342291163492, k: 77
[31074 31252 30736 ...  3989  1602 13502]
Threshold: 15, Removal error: 1.1265388774098517, k: 73
[31074 31252 30663 ... 30081 31261 25539]
Threshold: 16, Removal error: 1.1281262722110248, k: 72
[30663 30852 30736 ...  3038 31265 12224]
Threshold: 17, Removal error: 1.1320919901163413, k: 67
[30111 29478 29768 ... 12216  4192  5311]
Threshold: 18, Removal error: 1.1292014676989715, k: 62
[29768 29478 30111 ...  2029 30689 10639]
Threshold: 19, Removal error: 1.1256832061426556, k: 58
[30111 29768  3921 .

In [125]:
def k_svd (X, k):

    #Use the singular value decomposition numpy function to calculate the matrices U, Vt and the vector S of the eigenvalues of X 
    [U, S, Vt] = np.linalg.svd (X, full_matrices = False)

    #Consider the matrix U_k taking the first k columns of U
    U_k = U [:,0:k]
    #Consider the matrix Vt_k taking the first k rows of Vt
    Vt_k = Vt [0:k,:]
    #Consider the diagonal matrix Sigma_k taking the first k eigenvalues of X
    Sigma_k = np.diag (S[0:k])
    #Calculating the k-truncated svd matrices X_k
    X_k = np.dot ( U_k,  np.dot (Sigma_k, Vt_k) )

    return X_k

def detect_outliers (X, k, c):
    #create the k-truncated svd of X
    X_k = k_svd (X, k)

    #calculate the vector of the norm of the difference of the rows between X and X_k
    error = np.linalg.norm ( X - X_k, axis = 1 )

    #calculate the threshold
    #determine the mean of the vector of the error
    m = np.mean (error)
    #determine the variance of the vector of the error
    var = np.var ( error )
    #determine the threshold as the sum between the mean and 3 time the variance of the vector of the error
    threshold = m + c * var

    #determine the outlier as the indices where the error is above the threshold
    outlier = np.where (error > threshold)
    #redimension the dimension of the array
    outlier = np.array(outlier).flatten()

    return outlier

[X1, y1] = load_Xy ('data.csv')
n = X1.shape[0]
[X2,y2] = load_Xy ('data2.csv')

In [126]:
def analysis (k, c, n):
    outlier = detect_outliers(X2, k, c)
    m = len (outlier)

    v = np.arange (29332, 31331)
    tot = 0
    for i in v:
        if np.all( np.isin(i, outlier) ):
            tot = tot + 1

    a = tot / 2000
    b = ( m - tot ) / m
    d = ( m - tot ) / n

    #print ( "true outliers spotted: {}, fake outliers: {}, modified non-outliers: {}, dimension: {}".format (a, b, d, len (outlier)) )

    return [a, b, d, m]

analysis (83, 3, X1.shape[0])

[0.896, 0.15351913084553614, 0.011080049093140597, 2117]

In [127]:
def c_choice (k, n):
    tot_fake = 1
    c = 1
    j = 1

    while j < 20:
        [a,  b, d, m] = analysis (k, j, n)

        if m >= 2000:
            if (b < tot_fake):
                c = j
                tot_fake = b
        
        j = j+1
    
    return c-1

In [128]:
for k in [ 86-i for i in range (1,20) ]:
    c = c_choice (k, n)
    outlier = detect_outliers (X2, k, c)

    X_k = k_svd (X2, k)

    error_1 = np.linalg.norm ( X1 - X_k[0:n, :], ord = 'fro' ) / np.linalg.norm ( X1, ord = 'fro' )

    print("k: {}, error_1: {}, c: {}, dimension: {}".format(k, error_1, c, len (outlier) ) )

KeyboardInterrupt: 

In [ ]:
#the value k = 83 and c = 3 are considered to us valuable
def outlier_removal (X2, y2, k, c):
    outlier = detect_outliers (X2, k, c)
    X_k = k_svd (X2, k)

    X = np.delete (X_k, outlier, axis = 0)
    y = np.delete (y2, outlier, axis = 0)

    return X,y,len(outlier)

This function returns the percentage of correctly classified points in the test sets after training. It does so for the original dataset $X_1,y_1$ and the dataset $X_2, y_2$ after removing its outliers using the function outlier_removal. It also returns the number of removed outliers from $X_2,y_2$.

The inputs are $X_1,y_1$, $X_2,y_2$, a list of hyperparameters for logistic regression and values for the truncation of $X_2,y_2$.

In [ ]:
#hyperparameters
hyper = [0.0001, 0.3, 0.01, 4000]

#values for the truncation of X2,y2
outlier_coeff = [83, 3]

#comparison function
def w_test_comparison(X1,y1,X2,y2,hyper, outlier_coeff):
    #unpacks hyperparameters and values for the k-truncation
    a,l,thr,N = hyper
    k,c = outlier_coeff

    #makes a random 50/50 split of our original data set
    X_train, y_train, X_test, y_test = train_test_split(X1,y1,0.5)

    #removes outliers from the dataset with fake datapoints, and makes a random 50/50 split
    A,b,m = outlier_removal(X2,y2, k,c)
    A_train, b_train, A_test, b_test = train_test_split(A,b,0.5)

    #runs logistic regression with sparse linear algebra for both the original training set and the training set with removed outliers
    w = lr_sparse_thr(X_train, y_train, a,l,thr, N); w_A = lr_sparse_thr(A_train, b_train, a,l,thr, N)

    #Calculates the percentage of correctly classified points after calculating the optimal weight vectors in logistic regression.
    acc = count_correctly_classified(X_test, y_test, w[0])/y_test.shape[0]*100
    acc_A = count_correctly_classified(A_test, b_test, w_A[0])/b_test.shape[0]*100
    return m, acc, acc_A
m, acc, acc_A = w_test_comparison(X1,y1,X2,y2,hyper, outlier_coeff)
print("Test accuracy for X1: {}%, and for X2 after removal of {} outliers: {}%". format(acc, m, acc_A))


Test accuracy for X1: 95.84753852447838%, and for X2 after removal of 2117 outliers: 95.31078860898138%
